In [1]:
# Set display width, load packages, import symbols
ENV["COLUMNS"]=72
using Distributions
using Interpolations
using Knet: Knet, dir, accuracy, progress, sgd, load143, save, gc, Param, KnetArray, Data, minibatch, nll, relu, training, dropout,sigm # param, param0, xavier_uniform
using Knet
using Images
using Plots
using LinearAlgebra
using IterTools: ncycle, takenth
using MLDatasets
using Base.Iterators: flatten
import CUDA # functional
using ImageTransformations
using Statistics
using Memento
using NPZ
# using Interpolations
using Random
atype=(CUDA.functional() ? KnetArray{Float32} : Array{Float32})

const F = Float32
params = Knet.params

cd("..")
include("PlotUtility.jl")
include("ImageUtility.jl")
include("TrainUtility.jl")
include("LayerUtility.jl")
include("LossUtility.jl")

using .PlotUtility
using .ImageUtility
using .TrainUtility
using .LayerUtility
using .LossUtility

########################### CHANGE THIS LINE FOR DATASET PARAMETER ##############################
dataset_name = "mnist"
exp_number = 1
########################### CHANGE THIS LINE FOR RESULT FOLDER NAME #############################
notebook_name = "Ablation_Study_Initial_Point" * "_" * dataset_name * string(exp_number)

use_saved_data = true
nc = nothing

if dataset_name == "mnist"
    nc = 1
    if use_saved_data
        xtrn = npzread("Data/MNIST_Train_Images.npy")
        xtrn = permutedims(xtrn, (3,4,2,1))
        xtst = npzread("Data/MNIST_Test_Images.npy")
        xtst = permutedims(xtst, (3,4,2,1))

    else

        xtrn,_ = MNIST.traindata()
        xtst,_ = MNIST.testdata()
        xtrn = Array{Float64, 3}(xtrn)
        xtst = Array{Float64, 3}(xtst)

        xtrn = resize_MNIST(xtrn, 32/28)
        xtst = resize_MNIST(xtst, 32/28)
        
    end
    
elseif dataset_name == "fashion"
    nc = 1
    if use_saved_data

        xtrn = npzread("Data/Fashion_MNIST_Train_Images.npy")
        xtrn = permutedims(xtrn, (3,4,2,1))
        xtst = npzread("Data/Fashion_MNIST_Test_Images.npy")
        xtst = permutedims(xtst, (3,4,2,1))
        
    else
        
        xtrn,_ = FashionMNIST.traindata()
        xtst,_ = FashionMNIST.testdata()
        xtrn = Array{Float64, 3}(xtrn)
        xtst = Array{Float64, 3}(xtst)

        xtrn = resize_MNIST(xtrn, 32/28)
        xtst = resize_MNIST(xtst, 32/28)

    end
    
elseif dataset_name == "cifar"
    nc = 3
    xtrn,_= CIFAR10.traindata()
    xtst,_ = CIFAR10.testdata()
    xtrn = Array{Float64, 4}(xtrn)
    xtst = Array{Float64, 4}(xtst)

elseif dataset_name == "coil"
    coil_path = "Data/coil20"
    coil = load_coil_dataset(coil_path);
    nc = 1
    dataset_size = size(coil,3)
    trn_perc = 0.8
    trn_size = Int(dataset_size * trn_perc)
    random_permutation = randperm(dataset_size)
    trn_idx = random_permutation[1:trn_size]
    tst_idx = random_permutation[trn_size + 1:end]
    xtrn = coil[:,:,trn_idx]
    xtrn = resize_gray_image_tensor(xtrn, 1/4)
    xtst = coil[:,:,tst_idx];
    xtst = resize_gray_image_tensor(xtst, 1/4)

end

batch_size = 64

dtrn = minibatch(xtrn, batch_size; xsize = (32, 32, nc,:), xtype = atype, shuffle = true)
dtst = minibatch(xtst, batch_size; xsize = (32, 32, nc,:), xtype = atype);

function weights(nc, nz,  ngf) 
        
    # Decoding Weights
    theta = [] # z -> x
    
    w, b = deconv_weight_init(4, 4, nz, ngf * 4;bias = true, return_param = false)
    push!(theta, w)
    push!(theta, b)
    push!(theta, (bnparams(ngf * 4))) # Batch Normalization
    
    w, b = deconv_weight_init(4, 4, ngf * 4, ngf * 2;bias= true, return_param = false)
    push!(theta, w)
    push!(theta, b)
    push!(theta, (bnparams(ngf * 2)))
    
    w, b = deconv_weight_init(4, 4, ngf * 2, ngf;bias = true, return_param = false)
    push!(theta, w)
    push!(theta, b)
    push!(theta, (bnparams(ngf)))
    
    w,b = deconv_weight_init(4,4,ngf, nc;bias = true, return_param = false)
    push!(theta, w)
    push!(theta, b)

    theta = map(a->convert(atype,a), theta)
    
    return Param.(theta)
end

moments1 = bnmoments()
moments2 = bnmoments()
moments3 = bnmoments()

function decode(theta, z; batch_size = 64, training = true)
    
    z = deconv4(theta[1], z, mode = 1) .+ theta[2]
    z = mybatchnorm(z, moments1, theta[3]; training = training)
    z = Knet.elu.(z)
    
    z = deconv4(theta[4], z, mode = 1, stride = 2, padding = 1) .+ theta[5]
    z = mybatchnorm(z, moments2, theta[6]; training = training)
    z = Knet.elu.(z)
    
    z = deconv4(theta[7], z, mode = 1, stride = 2, padding = 1) .+ theta[8]
    z = mybatchnorm(z, moments3, theta[9]; training = training)
    z = Knet.elu.(z)
    
    z = deconv4(theta[10], z, mode = 1, stride = 2, padding = 1) .+ theta[11]
    x_hat = Knet.sigm.(z)
    return x_hat
    
end

# (Reconstruction Error := Mean Squared Error)
inner_loss(x, z) = mean(sum((decode(theta, z; batch_size = size(x,4)) - x).^2, dims = 3))

function loss_train(theta, x)
    x_hat =  decode_train(theta, x; batch_size = size(x,4))
    L = mean(sum((x_hat - x).^2, dims = 3))
end

function loss_train(theta, d::Data)
    total_loss = 0
    n_instance = 0
    for x in d
        total_loss += loss_train(theta, x) * size(x,4)
        n_instance += size(x,4)
    end
    
    total_loss /= n_instance
end

function decode_train(theta, x; batch_size = 64)
    origin = Param(atype(zeros(1, 1, nz, batch_size)))
    derivative_origin = @diff inner_loss(x,origin)
    dz = grad(derivative_origin, origin)
    z = -dz
    x_hat = decode(theta, z; batch_size = batch_size, training = true)
    return x_hat
end


decode_train (generic function with 1 method)

# Train With ELU

In [2]:
nz = 256
ngf = 16
# nc = 1

# first batch of the test dataset
x_test_first = first(dtst);

# define model weights
theta = weights(nc, nz,  ngf);

# Define Learning Rate and Number of Epochs
lr = 2*1e-4
n_epochs = 50

# optim = Knet.Adam(lr = lr, beta1 = 0.9, beta2 = 0.999)

# Specify the optimizer for each param
for p in params(theta)
    p.opt =  Knet.Adam(lr = lr, beta1 = 0.9, beta2 = 0.999)
end
# Initialize Empty Lists for both training and test losses
trn_loss_list0 = Float64[]
tst_loss_list0 = Float64[]

# RECORD INITIAL LOSS VALUES
epoch_loss_trn_ = loss_train(theta, dtrn)
epoch_loss_tst_ = loss_train(theta, dtst)
push!(trn_loss_list0, epoch_loss_trn_)
push!(tst_loss_list0, epoch_loss_tst_)

# Training Loop
for epoch in progress(1:n_epochs)
    for (i,x) in enumerate(dtrn)
        
        derivative_model = @diff loss_train(theta, x)
        
        for p in theta
            dp = grad(derivative_model, p)
            update!(value(p), dp, p.opt)
        end

    end
    
    epoch_loss_trn = loss_train(theta, dtrn)
    epoch_loss_tst = loss_train(theta, dtst)
    print(epoch_loss_trn)
    push!(trn_loss_list0, epoch_loss_trn)
    push!(tst_loss_list0, epoch_loss_tst)
end

┣▊                   ┫ [4.00%, 2/50, 00:29/12:09, 29.17s/i] 

0.0043266513

┣█▏                  ┫ [6.00%, 3/50, 00:49/13:35, 19.75s/i] 

0.0017653904

┣█▌                  ┫ [8.00%, 4/50, 01:09/14:21, 19.92s/i] 

...

0.0003927586

┣████████████████████┫ [100.00%, 50/50, 16:50/16:50, 22.73s/i] 

0.00037089133

┣████████████████████┫ [100.00%, 50/50, 17:13/17:13, 20.66s/i] 


0.0003658837

# Train with RELU

In [3]:
function decode(theta, z; batch_size = 64, training = true)
    
    z = deconv4(theta[1], z, mode = 1) .+ theta[2]
    z = mybatchnorm(z, moments1, theta[3]; training = training)
    z = Knet.relu.(z)
    
    z = deconv4(theta[4], z, mode = 1, stride = 2, padding = 1) .+ theta[5]
    z = mybatchnorm(z, moments2, theta[6]; training = training)
    z = Knet.relu.(z)
    
    z = deconv4(theta[7], z, mode = 1, stride = 2, padding = 1) .+ theta[8]
    z = mybatchnorm(z, moments3, theta[9]; training = training)
    z = Knet.relu.(z)
    
    z = deconv4(theta[10], z, mode = 1, stride = 2, padding = 1) .+ theta[11]
    x_hat = Knet.sigm.(z)
    return x_hat
    
end

nz = 256
ngf = 16
# nc = 1

# first batch of the test dataset
x_test_first = first(dtst);

# define model weights
theta = weights(nc, nz,  ngf);

# Define Learning Rate and Number of Epochs
lr = 2*1e-4
# n_epochs = 3

# optim = Knet.Adam(lr = lr, beta1 = 0.9, beta2 = 0.999)

# Specify the optimizer for each param
for p in params(theta)
    p.opt =  Knet.Adam(lr = lr, beta1 = 0.9, beta2 = 0.999)
end
# Initialize Empty Lists for both training and test losses
trn_loss_list1 = Float64[]
tst_loss_list1 = Float64[]

# RECORD INITIAL LOSS VALUES
epoch_loss_trn_ = loss_train(theta, dtrn)
epoch_loss_tst_ = loss_train(theta, dtst)
push!(trn_loss_list1, epoch_loss_trn_)
push!(tst_loss_list1, epoch_loss_tst_)

# Training Loop
for epoch in progress(1:n_epochs)
    for (i,x) in enumerate(dtrn)
        
        derivative_model = @diff loss_train(theta, x)
        
        for p in theta
            dp = grad(derivative_model, p)
            update!(value(p), dp, p.opt)
        end

    end
    
    epoch_loss_trn = loss_train(theta, dtrn)
    epoch_loss_tst = loss_train(theta, dtst)
    print(epoch_loss_trn)
    push!(trn_loss_list1, epoch_loss_trn)
    push!(tst_loss_list1, epoch_loss_tst)
end

┣▊                   ┫ [4.00%, 2/50, 00:21/08:55, 21.40s/i] 

0.009941836

┣█▏                  ┫ [6.00%, 3/50, 00:43/11:53, 21.40s/i] 

0.0037082562

┣█▌                  ┫ [8.00%, 4/50, 01:05/13:37, 22.60s/i] 

...

0.00070881064

┣████████████████████┫ [100.00%, 50/50, 18:09/18:09, 21.95s/i] 

0.0007333916

┣████████████████████┫ [100.00%, 50/50, 18:32/18:32, 22.23s/i] 


0.0006910065

# Train with SELU

In [4]:
function decode(theta, z; batch_size = 64, training = true)
    
    z = deconv4(theta[1], z, mode = 1) .+ theta[2]
    z = mybatchnorm(z, moments1, theta[3]; training = training)
    z = Knet.selu.(z)
    
    z = deconv4(theta[4], z, mode = 1, stride = 2, padding = 1) .+ theta[5]
    z = mybatchnorm(z, moments2, theta[6]; training = training)
    z = Knet.selu.(z)
    
    z = deconv4(theta[7], z, mode = 1, stride = 2, padding = 1) .+ theta[8]
    z = mybatchnorm(z, moments3, theta[9]; training = training)
    z = Knet.selu.(z)
    
    z = deconv4(theta[10], z, mode = 1, stride = 2, padding = 1) .+ theta[11]
    x_hat = Knet.sigm.(z)
    return x_hat
    
end

nz = 256
ngf = 16
# nc = 1

# first batch of the test dataset
x_test_first = first(dtst);

# define model weights
theta = weights(nc, nz,  ngf);

# Define Learning Rate and Number of Epochs
lr = 2*1e-4
# n_epochs = 3

# optim = Knet.Adam(lr = lr, beta1 = 0.9, beta2 = 0.999)

# Specify the optimizer for each param
for p in params(theta)
    p.opt =  Knet.Adam(lr = lr, beta1 = 0.9, beta2 = 0.999)
end
# Initialize Empty Lists for both training and test losses
trn_loss_list2 = Float64[]
tst_loss_list2 = Float64[]

# RECORD INITIAL LOSS VALUES
epoch_loss_trn_ = loss_train(theta, dtrn)
epoch_loss_tst_ = loss_train(theta, dtst)
push!(trn_loss_list2, epoch_loss_trn_)
push!(tst_loss_list2, epoch_loss_tst_)

# Training Loop
for epoch in progress(1:n_epochs)
    for (i,x) in enumerate(dtrn)
        
        derivative_model = @diff loss_train(theta, x)
        
        for p in theta
            dp = grad(derivative_model, p)
            update!(value(p), dp, p.opt)
        end

    end
    
    epoch_loss_trn = loss_train(theta, dtrn)
    epoch_loss_tst = loss_train(theta, dtst)
    print(epoch_loss_trn)
    push!(trn_loss_list2, epoch_loss_trn)
    push!(tst_loss_list2, epoch_loss_tst)
end

┣▊                   ┫ [4.00%, 2/50, 00:24/10:05, 24.20s/i] 

0.0035211989

┣█▏                  ┫ [6.00%, 3/50, 00:48/13:14, 23.44s/i] 

0.001607066

┣█▌                  ┫ [8.00%, 4/50, 01:11/14:42, 22.91s/i] 

...

0.00042172044

┣████████████████████┫ [100.00%, 50/50, 18:51/18:51, 23.15s/i] 

0.00042650904

┣████████████████████┫ [100.00%, 50/50, 19:15/19:15, 23.11s/i] 


0.00041813828

In [10]:
title = ""
figsize = (1000,750)
epochs = 1:n_epochs

plot(epochs,trn_loss_list0[2:end] ,label = "ELU", xlabel = "Epochs", ylabel = "Loss",
title = title, minorgrid = true, xtickfontsize = 18,
ytickfontsize = 18, legendfontsize = 18, xguidefontsize=18, 
yguidefontsize=18, titlefontsize = 20, size = figsize, thickness_scaling = 1, tickfontsize=20,linewidth=5)

plot!(epochs,trn_loss_list1[2:end]  ,label = "RELU", xlabel = "Epochs", ylabel = "Loss",
title = title, minorgrid = true, xtickfontsize = 18,
ytickfontsize = 18, legendfontsize = 18, xguidefontsize=18, 
yguidefontsize=18, titlefontsize = 20, size = figsize, thickness_scaling = 1, tickfontsize=20,linewidth=5)

plot!(epochs,trn_loss_list2[2:end]  ,label = "SELU", xlabel = "Epochs", ylabel = "Loss",
title = title, minorgrid = true, xtickfontsize = 18,
ytickfontsize = 18, legendfontsize = 18, xguidefontsize=18, 
yguidefontsize=18, titlefontsize = 20, size = figsize, thickness_scaling = 1, tickfontsize=20,linewidth=5)

fig_name = "Ablation_Study_Activation"
savefig(joinpath("Ablation_Studies",fig_name))